# Les Optimisations en pétrole et chimie.

## Introduction.

## Sommaire 

- 1. <b>Simple Maximisation de profit en raffinerie.</b>
        - Notre problème de base de Raffinerie
        - Modélisation mathématique
        - Solution avec Python Pulp
- 2. <b>Allocation de flux à des échangeurs thermiques.</b>
        - Le problème de Prathamesh Mohite
        - Modélisation mathématique
        - Solution avec Python Pulp



## 1. <b>Simple Maximisation de profit en raffinerie.</b>

Notre problème de base de Raffinerie

Source : https://www.youtube.com/watch?v=Ht1atCXPp24
AP Monitor : Oil Refinery Optimization

- Je suis manager d'une raffinerie.
- J'ai deux fournisseurs qui vendent du pétrole Brut 24$ et 15$ respectivement.
- Ma raffinerie revends le gazoline 36$, le kérosène 24$ , Le fuel 21$ et les résidus 10$


Les caractéristiques de transformation sont exprimées dans le tableau suivant :
<div style="text-align:center">
<img src="img/raffinerie.png">
</div>
On voit que le pétrole BRUT 1 (Crude) permet de produire beaucoup plus de gazoil, par exemple.
Par contre, le pétrole BRUT 2 permet de produire plus de fuel, cela est exprimé en pourcentages.
On voit aussi la production maximum exprimée en BBL dans la colonne de droite et le coût de production par BBL, un SI UNIT des états unis qui signifie Barril de brut : "The abbreviation BBL stands for a barrel of crude oil."

Pour la fonction objectif, on sait qu'on veut maximiser notre profit , c'est l'inverse de réduire les coûts dans ce cas.

La modélisation : 
<div style="text-align:center">
<img src="img/raffinerie2.png">
</div>

Et si on veut simplifier la fonction objectif, on soustrait les coûts de production et d'achat à l'avance dans la fonction objectif : 

<div style="text-align:center">
<img src="img/raffinerie3.png">
</div>


## Solution avec Python Pulp

Avez Python Pulp, on voit qu'on obtient le même résultat que dans la vidéo de AP Monitor, qui utilise pourtant un autre solveur...

In [7]:
# Importer la librairie Pulp 

from pulp import *

# Créer un programme linéaire de maximisation
Mon_Probleme = LpProblem('maximisation_de_profit_raffinerie', LpMaximize)  

# Créer les variables du problème, 
# Les variables de décision se comptent en barils,
# du coup ce sont des variables entières car on ne peut pas avoir un demi baril.
X1 = LpVariable("Crud1", 0, None, LpInteger)   
X2= LpVariable("Crud2", 0, None, LpInteger)   


# Ecrire la fonction objectif à maximizer 
Mon_Probleme +=  8.1 * X1 + 10.2 * X2

# Les contraintes : 

# Production maximum exprimée en nombre de barils.

# Gazoil
Mon_Probleme += 0.80 * X1 + 0.44 * X2  <= 24000
# Kerosene
Mon_Probleme += 0.05 * X1 + 0.10 * X2  <= 2000
# Fuel
Mon_Probleme += 0.10 * X1 + 0.36 * X2  <= 6000

# Résoudre
Mon_Probleme.solve()
# On imprime les variables qui ont leur valeur optimisées
for v in Mon_Probleme.variables():
    print(v.name, "=", v.varValue)
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Profit total maximisé = ", value(Mon_Probleme.objective))

Crud1 = 26206.0
Crud2 = 6897.0
Profit total maximisé =  282618.0


# - 2. <b>Allocation de flux à des échangeurs thermiques.</b>

- Source : https://www.linkedin.com/pulse/crude-blending-gaining-competitive-edge-oil-gas-industry-mohite/
- Auteur : https://github.com/mohiteprathamesh1996/Heat-Exchanger-Assignment-/tree/main
- Auteur : Prathamesh Mohite

Si vous avez passé beaucoup de temps à travailler dans une raffinerie de pétrole, il est probable que vous ayez rencontré ces structures gigantesques disposées systématiquement comme celles de la photo de couverture ci-dessus. En termes mécaniques, ce système est celui d'un échangeur de chaleur de type « calandre et tube » déployé principalement dans le but de transférer ou d'échanger de la chaleur d'un flux de fluide à haute température vers un autre flux de fluide à basse température ou vice versa.

Une disposition optimisée des échangeurs de chaleur peut permettre de rediriger un canal efficace pour les fluides industriels vers d'autres processus ou d'assurer une évacuation appropriée dans les plans d'eau adjacents sans provoquer d'impact environnemental significatif. A travers cet article, nous allons tenter de formuler un problème de programmation linéaire utilisant le module PuLP de Python pour attribuer systématiquement des flux de sorties de fluides provenant de sources multiples à un ensemble d'échangeurs de chaleur garantissant un gaspillage minimal de matières premières et empêchant le mélange des fluides dans le but de minimiser le gaspillage de matières premières. coût global d'installation.

Dans le tableau suivant, on voit les coûts de faire parvenir un flux N à un échangeur Z, 
Le but est de minimiser le coût global de flux, tout en s'assurant que chaque echangeur soies utilisé.

In [8]:

# On charge les librairies
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from pulp import *

# On charge le datasets puis on le visionne
df = pd.read_excel("datasets/Heat Exchanger Assignment Optimization.xlsx", sheet_name="Installation Costs")
df.set_index("Streams", inplace=True)
df

,Exchanger_1,Exchanger_2,Exchanger_3,Exchanger_4,Exchanger_5,Exchanger_6,Exchanger_7,Exchanger_8,Exchanger_9,Exchanger_10
Streams,,,,,,,,,,
Stream_1,949,1176,1073,1042,1137,999,993,944,1052,855
Stream_2,1127,1108,1163,1032,1181,1168,989,1180,841,889
Stream_3,1088,936,966,943,1034,1118,1091,1156,829,1110
Stream_4,920,917,1139,817,1154,1141,1150,821,826,969
Stream_5,907,832,905,834,1055,1135,986,852,857,813
Stream_6,1121,877,1026,878,861,1026,827,1053,1162,957
Stream_7,989,1102,1157,1175,963,957,941,1087,1041,1182
Stream_8,1060,1126,845,854,962,818,851,882,895,1172
Stream_9,1147,1168,1186,1183,983,1100,1101,905,1092,1051


# Créer un dictionnaire des variables de décision

Dans ce problème, nous définissons la variable de décision binaire, X(i, j) telle que

     X (i, j) = 1 si le flux 'i' est attribué à l'échangeur 'j' sinon 0 pour tous i∈ (1,..10) et j ∈ (1,…10)

In [13]:
# Create a list of stream indexes
# Créer une liste indexée des flux 
streams = df.index.to_list()
print("Streams : {} \n".format(streams))

# Create a list of indexes for exchangers
# Créer une liste indexée d'échangeurs
exchanger_stations = df.columns.to_list()
print("Heat Exchangers : {} \n".format(exchanger_stations))

# Create a dictionary of binary type decision variables
# Créer un dictionnaire de toutes les variables binaires possibles 
# Ce sont toutes les "associations" possibles entre échangeurs et flux !
# Ce sont des variables binaire de "selection" qui vont s'activer lorsque leur valeur est à 1.
var_dict = LpVariable.dicts(
    name="assign",
    indices=[(i, j) for i in streams for j in exchanger_stations], 
    lowBound=0,
    cat="Binary")

# Display first 5 decision variables
# Montrer les 5 premières variables de décision
[var_dict[(i,j)] for i in streams for j in exchanger_stations][:5]

Streams : ['Stream_1', 'Stream_2', 'Stream_3', 'Stream_4', 'Stream_5', 'Stream_6', 'Stream_7', 'Stream_8', 'Stream_9', 'Stream_10'] 

Heat Exchangers : ['Exchanger_1', 'Exchanger_2', 'Exchanger_3', 'Exchanger_4', 'Exchanger_5', 'Exchanger_6', 'Exchanger_7', 'Exchanger_8', 'Exchanger_9', 'Exchanger_10'] 



[assign_('Stream_1',_'Exchanger_1'),
 assign_('Stream_1',_'Exchanger_2'),
 assign_('Stream_1',_'Exchanger_3'),
 assign_('Stream_1',_'Exchanger_4'),
 assign_('Stream_1',_'Exchanger_5')]

# La fonction objectif

Plutôt que d'écrire directement la fonction objectif à minimiser en dur : 
Minimize, Z = 949*X(1,1) + 1176*X(1,2)+….+1055*X(10,9) + 1175*X(10,10)
Qui associe chaque flux à chaque échangeur et qui est donc très longue à écrire,
il utilise l'itération sur le tableau.

In [10]:
# Define the LP Minimization problem

# On déclare un problème de Minimisation avec Python Pulp
model = LpProblem("Minimize_Installation_Costs", LpMinimize)


# Adding the objective function 

# La fonction objectif, c'est comme si on écrivait toutes les solutions possibles 
# Minimize, Z = 949*X(1,1) + 1176*X(1,2)+….+1055*X(10,9) + 1175*X(10,10)
# Sauf qu'en itérant sur le tableau, cela est beaucoup plus rapide et pratique.
model += lpSum([df.loc[(i,j)]*var_dict[(i,j)] for i in streams for j in exchanger_stations])

# Les contraintes

In [11]:
# Each stream must be assigned to exactly 1 heat exchanger
# Chaque flux doit être assigné exactement à 1 seul échangeur
for i in streams:
    model += lpSum([var_dict[(i, j)] for j in exchanger_stations]) == 1

# Every heat exchanger must receive input from exactly 1 stream  
# Tous les échangeurs doivent recevoir en entrée un seul flux  
for j in exchanger_stations:
    model += lpSum([var_dict[(i, j)] for i in streams]) == 1

# La solution

In [12]:
model.solve()

if LpStatus[model.status]=="Optimal":
    optimal_soln = pd.DataFrame(index = streams, 
                                columns = exchanger_stations)
    for s in streams:
        for e in exchanger_stations:
            optimal_soln.loc[s, e] = value(var_dict[(s, e)])
    
print("Total installation cost : $ {}".format(value(model.objective)))

optimal_soln


# pd.DataFrame([(optimal_soln[optimal_soln[c]==1].index.values[0], c) for c in optimal_soln.columns],
#              columns = ["ConnectionFrom", "ConnectionTo"])


# Saving the optimal solution results
if LpStatus[model.status]=="Optimal":
    optimal_soln = pd.DataFrame(
[(v.name, v.varValue) for v in model.variables() if v.varValue==1],
                                columns=["Assignment", "Status"])

optimal_soln






Total installation cost : $ 8773.0


,Assignment,Status
0,"assign_('Stream_1',_'Exchanger_1')",1.0
1,"assign_('Stream_10',_'Exchanger_9')",1.0
2,"assign_('Stream_2',_'Exchanger_10')",1.0
3,"assign_('Stream_3',_'Exchanger_3')",1.0
4,"assign_('Stream_4',_'Exchanger_4')",1.0
5,"assign_('Stream_5',_'Exchanger_2')",1.0
6,"assign_('Stream_6',_'Exchanger_7')",1.0
7,"assign_('Stream_7',_'Exchanger_5')",1.0
8,"assign_('Stream_8',_'Exchanger_6')",1.0
9,"assign_('Stream_9',_'Exchanger_8')",1.0
